# imports

In [63]:
import requests
import pandas as pd

# acquire data

In [64]:
api_url = "https://opendata.maryland.gov/api/id/crti-ybyp.json?$select=*&$order=`:id`+ASC&$limit=3000&$offset=0"
response = requests.get(api_url)
data = response.json()

In [65]:
data[0]

{'sno': '1',
 'complaint': '232264',
 'complaint_description': 'concern of loud noise in the Cheverly area',
 'complaint_type': 'Other',
 'recieved_date': '2024-03-06',
 'incident_date': '2024-03-06',
 'county': "Prince George's",
 'incident_closed_date': '2024-03-08',
 'incident_status_desc': 'Incident Closed-Managed'}

In [66]:
type(data)

list

In [67]:
df = pd.DataFrame(data)

# clean data

In [68]:
backup_data = df.copy()

In [69]:
df.dtypes

sno                      object
complaint                object
complaint_description    object
complaint_type           object
recieved_date            object
incident_date            object
county                   object
incident_closed_date     object
incident_status_desc     object
incident_zip             object
dtype: object

In [70]:
df['recieved_date'] = pd.to_datetime(df['recieved_date'])
df['incident_date'] = pd.to_datetime(df['incident_date'])
df['incident_closed_date'] = pd.to_datetime(df['incident_closed_date'])

In [71]:
df.dtypes

sno                              object
complaint                        object
complaint_description            object
complaint_type                   object
recieved_date            datetime64[ns]
incident_date            datetime64[ns]
county                           object
incident_closed_date     datetime64[ns]
incident_status_desc             object
incident_zip                     object
dtype: object

# analyze data

In [72]:
# For the summaries we're running, we'll limit that to just complaints received in the last year?
year_ago_date = pd.Timestamp('2023-03-09')

last_year_df = df[df["recieved_date"] > year_ago_date].copy()
last_year_df['county'].value_counts()

county
Baltimore City        101
Anne Arundel           85
Prince George's        83
Frederick              62
Baltimore              57
Montgomery             35
Cecil                  25
Dorchester             17
Harford                17
Howard                 15
Allegany               15
Washington             14
Charles                12
Carroll                 9
Wicomico                8
Not Yet Determined      7
Worcester               7
St. Mary's              6
Garrett                 5
Caroline                3
Queen Anne's            2
Somerset                2
Talbot                  1
Kent                    1
Statewide               1
Name: count, dtype: int64

In [73]:
last_year_df['complaint_type'].value_counts()

complaint_type
Air                                                                                 166
Odor                                                                                140
Other                                                                               121
Fugitive Dust/Particulate Matter                                                     35
Smoke                                                                                28
Air, Fumes, Odor                                                                     20
Open Burning                                                                         13
Air, Odor                                                                            11
Fumes                                                                                11
Open Burning, Smoke                                                                   7
Air, Odor, Smoke                                                                      4
Air, Fugitive Dus

In [74]:
last_year_df['complaint_type'] = last_year_df['complaint_type'].str.split(', ')

In [75]:
# Now, apply this to our full dataset.
complaints_by_type = last_year_df.explode("complaint_type")

In [76]:
complaints_by_type['complaint_type'].value_counts()

complaint_type
Air                                   223
Odor                                  186
Other                                 126
Smoke                                  56
Fugitive Dust/Particulate Matter       45
Fumes                                  39
Open Burning                           30
Asbestos Complaint                      3
Air Pollutant Release                   2
Noise Complaint                         2
Non-Tidal Wetlands/Waterway             2
Suspected Operation without Permit      1
ARA AQCP non-regulated entity           1
Name: count, dtype: int64

In [77]:
last_year_by_type = complaints_by_type["complaint_type"].value_counts()

In [78]:
last_year_df[last_year_df['county'].isin(['Not Yet Determined', 'Statewide'])]

,sno,complaint,complaint_description,complaint_type,recieved_date,incident_date,county,incident_closed_date,incident_status_desc,incident_zip
70,71,222068,concern of air pollution from private contractor,[Other],2024-02-05,2024-02-05,Not Yet Determined,2024-02-05,Incident Closed-Managed,NaN
300,301,181543,Odor inside apartment.,[Air],2023-09-22,2023-09-21,Not Yet Determined,NaT,Incident Not Applicable to MDE,NaN
313,314,181510,Unknown,[Other],2023-09-08,2023-09-08,Not Yet Determined,2023-09-22,Incident Closed - No further action,NaN
355,356,181349,Lead,[Other],2023-08-07,2023-08-07,Not Yet Determined,2023-08-08,Incident Closed - No further action,NaN
357,358,181343,testing of fugitive matter from AC,[Other],2023-08-07,2023-08-07,Not Yet Determined,2023-08-07,Incident Closed-Managed,NaN
398,399,171205,Concerned with general poor ambient air qualit...,[Air],2023-06-28,2023-06-28,Statewide,2023-06-29,Incident Closed-Managed,NaN
402,403,171180,Automobile shop operating without machine.,[Other],2023-06-26,2023-06-26,Not Yet Determined,2023-07-07,Incident Closed-Managed,NaN
577,578,140705,Freon Release,[Air],2023-03-28,2023-03-28,Not Yet Determined,NaT,Referred to Outside Agency,NaN


In [79]:
invalid_county_rows = last_year_df[last_year_df['county'].isin(['Not Yet Determined', 'Statewide'])]
invalid_county_rows.to_csv('exported_data/invalid_counties.csv')

In [80]:
last_year_df = last_year_df[last_year_df['county'].isin(['Not Yet Determined', 'Statewide']) == False]
last_year_df['county'].value_counts()

county
Baltimore City     101
Anne Arundel        85
Prince George's     83
Frederick           62
Baltimore           57
Montgomery          35
Cecil               25
Harford             17
Dorchester          17
Howard              15
Allegany            15
Washington          14
Charles             12
Carroll              9
Wicomico             8
Worcester            7
St. Mary's           6
Garrett              5
Caroline             3
Queen Anne's         2
Somerset             2
Talbot               1
Kent                 1
Name: count, dtype: int64

In [81]:
last_year_by_county = last_year_df["county"].value_counts()

In [82]:
last_year_by_county

county
Baltimore City     101
Anne Arundel        85
Prince George's     83
Frederick           62
Baltimore           57
Montgomery          35
Cecil               25
Harford             17
Dorchester          17
Howard              15
Allegany            15
Washington          14
Charles             12
Carroll              9
Wicomico             8
Worcester            7
St. Mary's           6
Garrett              5
Caroline             3
Queen Anne's         2
Somerset             2
Talbot               1
Kent                 1
Name: count, dtype: int64

# export data

In [83]:
last_year_by_type.to_csv("exported_data/complaint_typef_frequency.csv")
last_year_by_county.to_csv("exported_data/county_frequency.csv")

In [84]:
last_year_df.to_csv("exported_data/all_complaints.csv", index=False)